In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
current_dir = os.path.abspath('')
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
sys.path.append(parent_dir)



import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style


In [2]:
import json
import pandas as pd

In [4]:
CHESS_PRED_FP = "/home/chunyu/Projects/TestSuiteEval-main/datasets/bird/chess_pred.json"
BIRD_DAIL2_FP = "/home/chunyu/Projects/Dockers/autotest/.datasets/benchmarks/bird_dail.jsonlines"

In [5]:
from sqlglot import parse_one

In [20]:
df = pd.read_json(BIRD_DAIL2_FP, lines= True)
df.head(1)

,benchmark,index,schema,constraint,pair
0,california_schools,1,"{'FRPM': {'CDSCODE': 'VARCHAR', 'ACADEMIC YEAR...","[{'primary': [{'value': 'FRPM__CDSCODE'}]}, {'...",[SELECT `Free Meal Count (K-12)` / `Enrollment...


In [6]:
pred = json.load(open(CHESS_PRED_FP))

In [9]:
with open("/home/chunyu/Projects/Dockers/autotest/.datasets/benchmarks2/sqlsolver/bird/chess/sql2.sql", 'w') as fp:
    for row in pred:
        p =row['pred']

        try:
            p = parse_one(p, dialect = 'sqlite').sql(dialect='mysql')
        except Exception as e:
            ...
        finally:
            fp.write(p + '\n')
    

In [16]:
for index, row in df.iterrows():
    pair = row['pair']
    p = pred[row['index'] - 1]['pred']
    try:
        pair[1] = parse_one(p, dialect = 'sqlite').sql(dialect='mysql')
    except Exception as e:
        pair[1] = p
        ...
    finally:
        df.at[index, 'pair'] = pair  # Update value


In [22]:
df.to_json("/home/chunyu/Projects/Dockers/autotest/.datasets/benchmarks/bird_chess_verieql.jsonlines", lines= True, orient= 'records')

In [1]:
from sqlglot import parse_one, exp

In [ ]:
import sqlite3

def count_total_rows(database_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # Get the list of all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    total_rows = 0
    # Iterate through each table and count rows
    for table in tables:
        table_name = table[0]
        cursor.execute(f"SELECT COUNT(*) FROM `{table_name}`")
        count = cursor.fetchone()[0]
        total_rows += count

    # Close the connection
    conn.close()

    return total_rows

import os
def get_all_files(folder_path):
    files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) ]
    return files
# Example usage
# folder_path = '/home/chunyu/Projects/ParSEval/results/dail/2025-01-31_10-04-20'
# files = get_all_files(folder_path)
# print("Files:", len(files))

### Tree Structure

In [2]:
from collections import defaultdict

class ConstraintGroup:
    """Represents a grouped set of constraints over array elements."""
    def __init__(self):
        self.groups = defaultdict(set)  # Maps constraint types to indexed elements

    def add_constraint(self, var, condition):
        """Groups constraints by condition."""
        self.groups[condition].add(var)

    def get_factorized_constraints(self):
        """Returns merged constraints in generalized form."""
        return [f"{condition} for {vars}" for condition, vars in self.groups.items()]

    def __repr__(self):
        return str(self.get_factorized_constraints())

In [4]:
cg = ConstraintGroup()
cg.add_constraint("a0", "> 30")
cg.add_constraint("a1", "> 30")
cg.add_constraint("a2", "< 30")
cg.add_constraint("a0", "== 5")
cg.add_constraint("a2", "== 5")

print(cg.get_factorized_constraints())

["> 30 for {'a0', 'a1'}", "< 30 for {'a2'}", "== 5 for {'a0', 'a2'}"]


In [ ]:
class PathNode:
    def __init__(self, constraints=None, cluster=None):
        self.constraint_group = ConstraintGroup()  # Store constraints in grouped form
        if constraints:
            for var, cond in constraints:
                self.constraint_group.add_constraint(var, cond)
        
        self.cluster = cluster  # Execution path category
        self.children = []

    def add_child(self, constraints, cluster):
        """Adds a child node, merging its constraints."""
        child = PathNode([], cluster)
        child.constraint_group.groups = self.constraint_group.groups.copy()  # Copy parent constraints
        
        for (var, cond) in constraints:
            child.constraint_group.add_constraint(var, cond)  # Add new constraints

        self.children.append(child)
        return child

    def __repr__(self):
        return f"Node(Constraints={self.constraint_group}, Tag={self.cluster.tag}, Priority={self.cluster.priority})"

class Cluster:
    """Represents a group of execution paths sharing the same tag."""
    def __init__(self, tag, priority):
        self.tag = tag
        self.priority = priority
        self.nodes = []  # List of nodes in this cluster

    def add_node(self, node):
        """Adds a node to the cluster."""
        self.nodes.append(node)

    def __repr__(self):
        return f"Cluster(Tag={self.tag}, Priority={self.priority}, Nodes={len(self.nodes)})"

class PathTree:
    def __init__(self):
        self.root = PathNode([(None, "True")], Cluster("Root", 0))  # Default root node
        self.clusters = {}  # Dictionary of Cluster objects

    def create_cluster(self, tag, priority):
        """Creates a new cluster and saves it."""
        self.clusters[tag] = Cluster(tag, priority)

    def get_cluster(self, tag):
        """Retrieves an existing cluster by tag."""
        return self.clusters.get(tag, None)

    def traverse(self, node=None, depth=0):
        """Recursively prints the tree structure."""
        node = node if node else self.root
        print("  " * depth + f"{node.constraint_group} [Tag: {node.cluster.tag}, Priority: {node.cluster.priority}]")
        for child in node.children:
            self.traverse(child, depth + 1)

    def get_highest_priority_path(self):
        """Finds the next execution path based on cluster priorities."""
        best_cluster = max(self.clusters.values(), key=lambda c: c.priority, default=None)
        return best_cluster.nodes[0] if best_cluster and best_cluster.nodes else None
    
    def get_best_execution_path(self):
        """Finds the most promising execution path using constraint uniqueness."""
        queue = [self.root]
        best_node = None
        max_unique_constraints = 0

        while queue:
            node = queue.pop(0)
            unique_constraints = len(node.constraint_group.groups)  # Count unique constraints
            if unique_constraints > max_unique_constraints:
                max_unique_constraints = unique_constraints
                best_node = node
            queue.extend(node.children)

        return best_node  # Return path with the most unique constraints

In [15]:
tree = PathTree()

# Create execution clusters
tree.create_cluster("Critical", 3)
tree.create_cluster("Boundary", 2)
tree.create_cluster("Normal", 1)

# Get clusters
critical = tree.get_cluster("Critical")
boundary = tree.get_cluster("Boundary")

# Root node
x_positive = tree.root.add_child([("a0", "> 30"), ("a1", "> 30"), ("a2", "> 30")], critical)
x_negative = tree.root.add_child([("a0", "<= 30"), ("a1", "<= 30")], boundary)

# Further branches
y_large = x_positive.add_child([("a1", "== 5")], boundary)

# Print tree
tree.traverse()

# Get best execution path
best_path = tree.get_best_execution_path()
print(f"\nNext best path to execute: {best_path}")

['True for {None}'] [Tag: Root, Priority: 0]
  ['True for {None}', "> 30 for {'a2', 'a0', 'a1'}"] [Tag: Critical, Priority: 3]
    ['True for {None}', "> 30 for {'a2', 'a0', 'a1'}", "== 5 for {'a1'}"] [Tag: Boundary, Priority: 2]
  ['True for {None}', "<= 30 for {'a0', 'a1'}"] [Tag: Boundary, Priority: 2]

Next best path to execute: Node(Constraints=['True for {None}', "> 30 for {'a2', 'a0', 'a1'}", "== 5 for {'a1'}"], Tag=Boundary, Priority=2)


In [17]:
tree.get_cluster('Critical')

Cluster(Tag=Critical, Priority=3, Nodes=0)

In [ ]:
from sqlglot import parse_one

from sqlglot.optimizer import normalize
normalize.normalize(parse_one('not (a or b)'), dnf= True)

Not(
  this=Paren(
    this=Or(
      this=Column(
        this=Identifier(this=a, quoted=False)),
      expression=Column(
        this=Identifier(this=b, quoted=False)))))

### Plot Branches

In [17]:
BRANCH_PATH = "/home/chunyu/Projects/ParSEval/results/dail/2025-01-31_10-04-20/branch.log"

In [18]:
import ast
df = []
err = []
with open(BRANCH_PATH) as fp:
    for line in fp.readlines():
        try:
            line = line.replace('set()', '[]')
        # print(line)
            df.append(ast.literal_eval(f"""{line}"""))
        except Exception as e:
            err.append(line)
            print(line)
len(err)

0

### Plot Support Ratio and End to End Performance

In [21]:
METRIC_PATH = "/home/chunyu/Projects/ParSEval/results/dail/2025-01-31_10-04-20/metric.log"
import ast
metric_df = []
err = []
with open(METRIC_PATH) as fp:
    for line in fp.readlines():
        try:
            line = line.replace('set()', '[]')
        # print(line)
            metric_df.append(ast.literal_eval(f"""{line}"""))
        except Exception as e:
            err.append(line)
            print(line)
len(err)

0

In [24]:
data_gen_time = 0
cnt = 0
for row in metric_df:
    if row['state'] in ['SUCCESS', 'TIMEOUT']:
        data_gen_time += row['data_gen']
        cnt += 1

data_gen_time / cnt


85.52000434526413

In [26]:
len(tuple(row for row in metric_df if row['state'] in ['SUCCESS', 'TIMEOUT']))

662

In [44]:
parseval_complete_cnt = 0
for row in df:
    parseval_complete_cnt += 2 ** len(row['negatives'])
parseval_complete_cnt / len(df)

13.512323943661972

In [45]:
parseval_operator_cnt = 0
for row in df:
    ops = set()
    for b in row['negatives']:
        if b.startswith('NEG_o'):
            ops.add(b[:6])
    parseval_operator_cnt += len(ops)
parseval_operator_cnt / len(df)

1.852112676056338

### Plot SMT Complexity

In [2]:
SMT_FP = "/home/chunyu/Projects/ParSEval/results/dail/2025-02-13_16-01-36/smt.log"

In [ ]:
import ast
smt_df = []
total_nodes_in_smt = 0
with open(SMT_FP) as fp:
    for line in fp.readlines():
        row = line
        # row = ast.literal_eval(line)
        # 'complexity': 248}
        smt_df.append(row)
        total_nodes_in_smt += int(row[row.rindex(':') + 1 : row.rindex('}')])

total_nodes_in_smt / len(smt_df)


1076.5

ImportError: cannot import name 'tarfile' from 'backports' (/home/chunyu/miniconda3/envs/python38/lib/python3.8/site-packages/backports/__init__.py)